In [ ]:
#---------------------------------------------------------------------------------------------GENERAL-----------------------------------------------------------------------------------------------
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import VOCDetection
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.multiprocessing as mp
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torch.cuda.amp import autocast, GradScaler
from typing import Dict
import numpy as np
from torch.utils.data import Dataset
import os
from PIL import Image
import collections
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch.nn as nn
import torchviz
from torchsummary import summary
import re
import cv2  
import easyocr
import math
from datetime import datetime
import _tkinter
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageSequence
import cv2
import numpy as np
from mss import mss
import threading
import time
import pyttsx3
from docx import Document
from scipy.interpolate import interp1d
import pyautogui
import gc

#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))     

In [ ]:
# Load the pretrained model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# Replace the classifier with a new one
num_classes = 6
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Put model to device
device = torch.device('cpu')
model.to(device)

# Load saved model
model.load_state_dict(torch.load('ui/fasterrcnn/model.pt'))
model.eval()

# Data processing
data_transforms = transforms.Compose([ 
    transforms.Resize(512),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# this function finds the average difference between all the elements in a list, then adds a new element at the end
# equal to the last element minus the average
def filter_coordinates(pixel_coordinates, threshold=100):
    horizontal_lines = {}
    vertical_lines = {}

    # Check the shape of the input and reshape if needed
    if pixel_coordinates.shape[1] == 1 and pixel_coordinates.shape[2] == 2:
        pixel_coordinates = pixel_coordinates.reshape(-1, 2)

    # Separate coordinates into horizontal and vertical lines
    for coord in pixel_coordinates:
        x, y = int(coord[0]), int(coord[1])
        if y not in horizontal_lines:
            horizontal_lines[y] = []
        if x not in vertical_lines:
            vertical_lines[x] = []
        horizontal_lines[y].append(x)
        vertical_lines[x].append(y)

    # Filter horizontal lines longer than the threshold
    for y, x_values in horizontal_lines.items():
        if len(x_values) > threshold:
            pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 0], x_values)), :]

    # Filter vertical lines longer than the threshold
    for x, y_values in vertical_lines.items():
        if len(y_values) > threshold:
            pixel_coordinates = pixel_coordinates[np.logical_not(np.isin(pixel_coordinates[:, 1], y_values)), :]

    return pixel_coordinates

def remove_corner_pixels(coordinates, width, height):
    # Define a function to count pixels near corners
    def count_corner_pixels(coordinates, distance):
        count = 0
        for coord in coordinates:
            x, y = coord[0], coord[1]
            if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
                count += 1
        return count

    # Start with an initial distance
    distance = 10
    corner_pixels_count = count_corner_pixels(coordinates, distance)

    # Adjust the distance based on the count of pixels near corners
    if corner_pixels_count != 0:
        distance = 100 / corner_pixels_count

    filtered_coords = []
    for coord in coordinates:
        x, y = coord[0], coord[1]
        # Check if the pixel is in or within the calculated distance of the corners
        if ((x < distance or x >= width - distance) and (y < distance or y >= height - distance)):
            continue
        filtered_coords.append(coord)
        
    return np.array(filtered_coords)

def calculate_new_list(lst):
    # Calculate the differences between successive elements
    differences = [lst[i] - lst[i + 1] for i in range(len(lst) - 1)]
    
    # Calculate the average difference
    avg_difference = sum(differences) / len(differences)
    
    # Add a new element at the end of the list
    new_element = lst[-1] - avg_difference 
    lst.append(new_element)

    return lst

def extract_currency(string):
    # Dictionary to map symbols to currency names
    currency_map = {
        'US': 'USD',
        '$': 'USD',
        '€': 'EUR',
        '£': 'GBP',
        '¥': 'JPY',
        'A$': 'AUD',
        'C$': 'CAD',
        'S': 'USD',   # Misidentification for $
        'E': 'EUR',   # Misidentification for €
        'L': 'GBP',   # Misidentification for £
        'Y': 'JPY',   # Potential misidentification for ¥
    }
    
    # Build a pattern that checks for each symbol followed by optional whitespace and a digit or a period
    pattern_elements = [re.escape(symbol) + r'\s*[0-9\.]' for symbol in currency_map.keys()]
    pattern_string = '|'.join(pattern_elements) + r'|USD|EUR|GBP|JPY|AUD|CAD'
    pattern = re.compile(pattern_string)
    
    match = pattern.search(string)
    
    if match:
        symbol = match.group()[:-1].strip()  # Exclude the trailing digit or period and any whitespace from the match
        return currency_map.get(symbol, symbol).lower()
    else:
        return None
def remove_last_if_not_digit(s):
    if s and not s[-1].isdigit():
        return s[:-1]
    return s

def extract_percentage(string):
    # Correct misidentifications within the string
    corrected_string = correct_misidentifications(string)
    # Find the percentage pattern in the corrected string
    match = re.search(r'([-+]?\d+(\.\d+)?)\s*%', corrected_string)
    if match:
        raw_value = match.group(1)
        raw_value = correct_decimal_point(raw_value)
        raw_value = remove_last_if_not_digit(raw_value)
        percentage = float(raw_value)
        return percentage
    return None

def extract_bracketed_timeframe(string):
    match = re.search(r'\(([^)]+)\)', string)
    if match:
        bracketed_string = match.group(1)
        return bracketed_string.lower()
    return None

def correct_value(text):
    if text and text[0].isdigit() and text[0] in ['S', '5']:
        return '$' + text[1:]
    return text

def extract_float_value(input_string):
    numbers = re.findall(r'\d+\.\d+', input_string)                                      # Find all the floating-point numbers in the string using a regular expression
    return float(numbers[0])                                                             # Return the first floating-point number found, as a float
    
def correct_misidentifications(text):
    corrections = {
        'O': '0',
        'G': '6',
        'B': '8',
        'I': '1',
        'S': '5',
        'Z': '2',
        'Q': '0',
        'T': '7',
        'L': '1',
        'A': '4'
    }
    
    corrected_text = text
    for char, replacement in corrections.items():
        corrected_text = corrected_text.replace(char, replacement)
    
    corrected_text = re.sub(r'[^\d.]+', '', corrected_text)  # Keep only numeric characters and decimal points
    return corrected_text

def correct_decimal_point(value):
    value_str = str(value)

    # Check if the first digit is '0' and the second character is not '.'
    if value_str[0] == '0' and (len(value_str) == 1 or value_str[1] != '.'):
        value_str = '0.' + value_str[1:]

    # If the original value was a number, return as a float; otherwise, return as a string
    if isinstance(value, (int, float)):
        return float(value_str)
    return value_str

def process_ocr_string(y_axis):
    values = y_axis.split(',')
    corrected_values = []
    removed_indices = []
    
    for index, value in enumerate(values):
        # If the value contains a space not at the beginning or end, replace with a decimal point
        value = re.sub(r'(?<=\S) (?=\S)', '.', value)
        
        # If the value contains both digits and characters
        if re.search(r'\d', value) and re.search(r'[a-zA-Z]', value):
            corrected_value = correct_misidentifications(value)
        else:
            corrected_value = value

        # If the corrected value is still a valid float, add it to the results
        try:
            corrected_values.append(float(correct_decimal_point(corrected_value)))
        except ValueError:
            removed_indices.append(index)  # Save the index of the value that was removed

    return corrected_values, set(removed_indices)

def time_mapping(input_string):
    mapping = {
        '1h': '1 hour',
        '4h': '4 hours',
        '24h': '1 day',
        '1d': '1 day',
        '7d': '7 days',
        '1w': '7 days',
        '1m': '1 month',
        '3m': '3 months',
        '6m': '6 months',
        '1y': '1 year',
        'ytd': 'year to date',
        'all': 'year to date'
    }
    return mapping.get(input_string, 'N/A')

def clean_memory():
    tensor = None
    torch.cuda.empty_cache()
    del tensor
    gc.collect()

In [ ]:
def resize_frame(frame):
    original_width, original_height = frame.size
    if original_width < original_height:
        new_width = 512
        aspect_ratio = original_height / original_width
        new_height = int(new_width * aspect_ratio)
    else:
        new_height = 512
        aspect_ratio = original_width / original_height
        new_width = int(new_height * aspect_ratio)
    return frame.resize((new_width, new_height))

def process_frame(frame_received, frame_type): 
    CUSTOM_CLASSES = {"name": 1, "value": 2, "x-axis": 3, "y-axis": 4, "plot":5}

    img = frame_received.convert('RGB')                      # Ensure the image is in RGB mode
    orig_width, orig_height = img.size                                   # Store the original image dimensions

    # Process the image
    img_processed = data_transforms(img)

    # Calculate the scale ratios
    new_width, new_height = img_processed.shape[1], img_processed.shape[2]

    # Calculate the scale ratios
    if orig_width <= orig_height:
        new_height = 512
        new_width = int(orig_width * (new_height / orig_height))
    else:
        new_width = 512
        new_height = int(orig_height * (new_width / orig_width))

    width_ratio = new_width / orig_width
    height_ratio = new_height / orig_height

    with torch.no_grad():
        prediction = model([img_processed])

    # Scale bounding boxes back to original image size
    for box in prediction[0]['boxes']:
        box[0] *= (1 / width_ratio)
        box[2] *= (1 / width_ratio)
        box[1] *= (1 / height_ratio)
        box[3] *= (1 / height_ratio)

    # Debugging
    #print(prediction[0]['boxes'])
    #print(prediction[0]['scores'])

    results = {}                                 # Dictionary to store the results
    yaxis_coordinates = []                       # Initialize a list to store the Y coordinates
    label5_image = None                          # Variable to store the cropped image for label 5
    y_axis_image = None

    reader = easyocr.Reader(lang_list=['en'])       # Create a reader to recognize the English language

    for box, label, score in zip(prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores']):
        if label >= 1 and label <= 4 and score > 0.7:                                                 # Check if the score is above 0.7 and label is between 1 and 4
            xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
            cropped_image = img.crop((xmin, ymin, xmax, ymax))                                        # Crop the image          
            bw_img = cropped_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
            bw_img_array = np.array(bw_img)

            # Resize the image (e.g., double the size)
            resized_img = cv2.resize(bw_img_array, (0,0), fx=10, fy=10, interpolation = cv2.INTER_CUBIC)

            # Contrast Enhancement
            clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(16, 16))
            enhanced_img = clahe.apply(resized_img)

            # Apply Otsu's thresholding
            _, thresh_img = cv2.threshold(enhanced_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            if label == CUSTOM_CLASSES["y-axis"]:
                y_axis_image = cropped_image

                result = reader.readtext(thresh_img)
                text_results = [item[1] for item in result]                            # Extract only the text
                text_found = ','.join(text_results)

                for (bbox, text, prob) in result:
                    bottom_y1 = bbox[1][1]  # Get the y-coordinate of one of the bottom points
                    bottom_y2 = bbox[2][1]  # Get the y-coordinate of the other bottom point
                    #print(bottom_y1, bottom_y2)
                    average_y = (bottom_y1 + bottom_y2) / 2  # Calculate the average y-coordinate
                    quarter_y = (bottom_y2 + average_y) / 2
                    yaxis_coordinates.append(int(quarter_y / 10))  # Add the average y-coordinate for each text box available (divide by the upscale factor)
            else:
                result = reader.readtext(thresh_img, detail = 0)      
                text_found = ','.join(result)
                text_found = re.sub('\n+', ',', text_found)                                           # Replace one or more consecutive newline characters with a single comma


            label_name = [name for name, id in CUSTOM_CLASSES.items() if id == label][0]              # Find the label name using the CUSTOM_CLASSES dictionary
            results[label_name] = text_found                                                          # Store the result in the dictionary

        elif label == 5 and score > 0.7:                                                              # Check if the label is 5 and the score is above 0.7
            xmin, ymin, xmax, ymax = [int(i) for i in box]                                            # Convert box coordinates to integers
            label5_image = img.crop((xmin, ymin, xmax, ymax))                                         # Crop the image for label 5
    # The results dictionary now contains the extracted text for labels 1 to 4
    #print(results)

    #############################################################################################################################
    # Plot processing
    bw_label5_img = label5_image.convert("L")                                                       # Convert the cropped image to grayscale (black and white)
    bw_label5_img_array = np.array(bw_label5_img)

    # Apply Gaussian blur to reduce noise and improve the result of Canny edge detection
    blurred_label5_img = cv2.GaussianBlur(bw_label5_img_array, (3, 3), 0)

    # First, apply a binary threshold on the image
    thresh_label5_img = cv2.adaptiveThreshold(blurred_label5_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    inverted_label5_img = cv2.bitwise_not(thresh_label5_img)

    # Define a kernel for the erosion (a 3x3 square is common)
    kernel = np.ones((2,2), np.uint8)
    closed_label5_img = cv2.morphologyEx(inverted_label5_img, cv2.MORPH_CLOSE, kernel)

    ######################################################################################################################
    # Find the contours
    contours, hierarchy = cv2.findContours(closed_label5_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate the perimeter for each contour and find the index of the longest one
    max_length = 0
    longest_contour_index = 0
    for i, contour in enumerate(contours):
        length = cv2.arcLength(contour, True) # The 'True' argument indicates that the contour is closed
        if length > max_length:
            max_length = length
            longest_contour_index = i

    # Extract the longest contour
    longest_contour = contours[longest_contour_index]

    height, width = np.zeros_like(closed_label5_img).shape
    #print("Plot size is: ", height,width)
    filtered_coordinates = filter_coordinates(longest_contour)
    filtered_coordinates = remove_corner_pixels(filtered_coordinates, width, height)

    # Finding the highest pixel (smallest y value)
    highest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmin()]

    # Finding the lowest pixel (largest y value)
    lowest_pixel = filtered_coordinates[filtered_coordinates[:, 1].argmax()]

    # Finding the rightmost pixel (largest x value), and among those, the one with the lowest y value
    right_most_pixels = filtered_coordinates[filtered_coordinates[:, 0] == filtered_coordinates[:, 0].max()]
    right_most_pixel = right_most_pixels[right_most_pixels[:, 1].argmin()]

    ######################################################################################################################
    # DRAWING THE Y-AXIS DATA LOCATION ON THE IMAGE
    y_axis_str = results['y-axis']
    y_axis_values, y_axis_val_rem_idx = process_ocr_string(y_axis_str)
    yaxis_coordinates = [value for index, value in enumerate(yaxis_coordinates) if index not in y_axis_val_rem_idx]
    #print(y_axis_values, len(y_axis_values))
    #print(yaxis_coordinates, len(yaxis_coordinates))
    #print(y_axis_val_rem_idx)
    y_axis_values_new = calculate_new_list(y_axis_values)
    yaxis_coordinates_new = calculate_new_list(yaxis_coordinates)
    yaxis_coordinates_new = list(map(round, yaxis_coordinates_new))

    #print("New Y-axis values:", y_axis_values_new)
    #print("New Y coordinates:", yaxis_coordinates_new)

    ######################################################################################################################
    # FIRST INTERPOLATION METHOD
    # Value interpolation from curve
    # Create a linear interpolation function
    f = interp1d(yaxis_coordinates_new, y_axis_values_new)

    try:
        type1_interp_current_val = round(float(f(right_most_pixel[1])), 5)
        #print(f"The Y-axis value for the Y coordinate {right_most_pixel[1]} is {type1_interp_current_val}")
    except ValueError:
        type1_interp_current_val = 1e9

    # value from plot
    value_str = correct_value(correct_decimal_point(results['value']))
    stock_value = extract_float_value(value_str)
    #print(f"The stock value from the plot is {stock_value}")

    # value difference/accuracy
    type1_pct_diff = abs(round(((stock_value - type1_interp_current_val) / stock_value) * 100, 5))
    #print(f"The percentage difference is {type1_pct_diff:.2f}%")

    ######################################################################################################################
    # SECOND INTERPOLATION METHOD
    y_axis_val_diff = abs(y_axis_values[-1] - y_axis_values[0])
    y_axis_coord_dif = abs(yaxis_coordinates[-1] - yaxis_coordinates[0])
    y_axis_val_per_pixel = y_axis_val_diff/y_axis_coord_dif

    #print(y_axis_val_per_pixel)

    plot_height = label5_image.height
    type2_interp_current_val = (abs(plot_height - right_most_pixel[1])) * y_axis_val_per_pixel

    #print(f"Curve value based on secondaly calculation is {type2_interp_current_val}")

    # value from plot
    type2_pct_diff = abs(round(((stock_value - type2_interp_current_val) / stock_value) * 100, 3))
    #print(f"The percentage difference is {type2_pct_diff:.3f}%")
    ######################################################################################################################
    # Compare the 2 methods and choose the one with the lowest percentage difference
    if type1_pct_diff < type2_pct_diff:
        lowest_value  = round(float(f(lowest_pixel[1])), 5)
        highest_value = round(float(f(highest_pixel[1])), 5)
    else:
        lowest_value  = round((abs(plot_height - lowest_pixel[1])) * y_axis_val_per_pixel, 5)
        highest_value = round((abs(plot_height - highest_pixel[1])) * y_axis_val_per_pixel, 5)

    ######################################################################################################################
    #print(user_choice)
    
    if user_choice == "plot_to_speech" or user_choice == "report_creation":
        current_date = datetime.now().date()
        human_readable_date = current_date.strftime('%dth of %B %Y').replace('%dth', str(current_date.day)
                              + {1: 'st', 2: 'nd', 3: 'rd'}.get(current_date.day % 10, 'th'))
        name = results['name'].split(" Price")[0]

        phrase = f"The current value for the currency {name} is {str(stock_value)}. \
                   This represents a drop of {str(extract_percentage(results['value']))}% \
                    in the timeframe of {time_mapping(extract_bracketed_timeframe(results['value']))}. \
                   The currency's lowest value in the same timeframe is {str(lowest_value)}, \
                   meanwhile the highest value is  {str(highest_value)}. \
                   These values are accurate as of {human_readable_date}"
        
        phrase = ' '.join(phrase.split())                                                    # Removing extra whitespace and newline characters                                                          
        #print(phrase)

        # close processing dialog window
        close_processing_dialog()
        
        if user_choice == "plot_to_speech":
            speak_text(phrase)
        else:
            if frame_type == "video":
                doc_path = 'system_outputs/report_video.docx'                  # Define the path to the document
            else:
                doc_path = 'system_outputs/report_image.docx'                  # Define the path to the document
                
            directory = os.path.dirname(doc_path)                    # Create the directory if it doesn't exist
            if not os.path.exists(directory):
                os.makedirs(directory)
                
            if os.path.exists(doc_path) and append_to_doc:                               # Check if the file exists and the append_to_doc variable is True       
                doc = Document(doc_path)                                                 # Open the existing document and append the new paragraph
                doc.add_paragraph(phrase)
            else:
                # Create a new document and add the paragraph
                doc = Document()
                doc.add_paragraph(phrase)
                
            # Save the document
            doc.save(doc_path)
        
    elif user_choice == "dataset_cration":
        # Define the observations
        data = {
            'name': results['name'].split(" Price")[0],
            'currency': extract_currency(results['value']),
            'value.pct.drop': extract_percentage(results['value']),
            'current.value': stock_value,
            'type1.interp.current.value': type1_interp_current_val,
            'type2.interp.current.value': type2_interp_current_val,
            'type1.curr.val.pct.diff': type1_pct_diff,
            'type2.curr.val.pct.diff': type2_pct_diff,
            'lowest.value': lowest_value,
            'highest.value': highest_value,
            'timeframe': time_mapping(extract_bracketed_timeframe(results['value'])),
            'date': datetime.now().date(),
            'time': datetime.now().time().strftime('%H:%M:%S')                            # Format the time properly
        }
        # Create a DataFrame
        df = pd.DataFrame(data, index = [0])
        
        currency = extract_currency(results['value'])
        percentage_drop = extract_percentage(results['value'])
        print(f"The raw OCR reading fro the currency is: {results['value']}, the corrected output is: {currency}")
        print(f"The raw OCR reading for the percentage drop is: {results['value']}, the corrected output is: {percentage_drop}")

        # Create the directory if it doesn't exist
        if frame_type == "video":
            dataset_path = "system_outputs/dataset_video.csv"
        else:
            dataset_path = "system_outputs/dataset_image.csv"
        directory = os.path.dirname(dataset_path)
        if not os.path.exists(directory):
            os.makedirs(directory)

        if os.path.exists(dataset_path) and frame_type == "video":                        # Check if the file exists and the append_to_file variable is True 
            df.to_csv(dataset_path, mode='a', header=False, index=False)                     # Append to the existing file (without writing the header again)
        else:
            df.to_csv(dataset_path, index=False)                                      # Save the DataFrame to a new CSV file (or replace the existing one)
            
        clean_memory()

In [ ]:
def on_press(event):
    global x, y
    x, y = event.x_root, event.y_root

def on_drag(event):
    global x, y
    deltax = event.x_root - x
    deltay = event.y_root - y
    new_x = transparent_window.winfo_x() + deltax
    new_y = transparent_window.winfo_y() + deltay
    transparent_window.geometry(f"{width}x{height}+{new_x}+{new_y}")
    x, y = event.x_root, event.y_root

def on_resize(event):
    global width, height
    width = event.x
    height = event.y
    transparent_window.geometry(f"{width}x{height}+{transparent_window.winfo_x()}+{transparent_window.winfo_y()}")

def start_capture(event):
    global capturing, left, top, width, height
    capturing = True
    left = transparent_window.winfo_x()
    top = transparent_window.winfo_y()
    threading.Thread(target=desktop_capture_process).start()
    
    transparent_window.destroy()  # Close the transparent window after capturing
    #show_processing_dialog()

def desktop_capture_process():
    #time.sleep(3)
    
    global left, top, width, height, capturing

    with mss() as sct:
        while capturing:  # Stop when capturing is False
            monitor = {"top": top, "left": left, "width": width, "height": height}
            capture = sct.grab(monitor)
            pil_frame = Image.frombytes("RGB", capture.size, capture.rgb)
            resized_frame = resize_frame(pil_frame)
            process_frame(resized_frame, "video")

    #close_processing_dialog()
    
def create_highlighted_window():
    global transparent_window, width, height
    width, height = 900, 600
    transparent_window = tk.Toplevel(root)
    transparent_window.overrideredirect(1) # Remove window decorations
    transparent_window.attributes('-alpha', 0.3) # Make the window transparent
    transparent_window.geometry(f"{width}x{height}+100+100")
    transparent_window.attributes('-topmost', True) # Keep the window always on top

    frame = tk.Frame(transparent_window, borderwidth=10, relief="solid")
    frame.pack(fill="both", expand=True)

    transparent_window.bind('<ButtonPress-1>', on_press)
    transparent_window.bind('<B1-Motion>', on_drag)
    transparent_window.bind('<B3-Motion>', on_resize)
    transparent_window.bind('<ButtonPress-2>', start_capture)

def desktop_capture():
    global capture_window, capture_canvas
    # Create the transparent window for selecting the capture area
    create_highlighted_window()

    
def open_system():
    root.withdraw()
    system_page.deiconify()
    
def open_options():
    root.withdraw()
    system_page.withdraw()
    options_page.deiconify()

def show_description():
    messagebox.showinfo("System Description", "System from charts to data conversion.")

def quit_program():
    root.destroy()

def choose_image():
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.bmp *.jpeg *.tiff")])
    if image_path == "":
        messagebox.showinfo("User information", "No file has been selected, the operation has been cancelled!")
        pass
    else:
        show_processing_dialog()
        # Start the processing task in a separate thread, passing the image path as an argument
        threading.Thread(target=process_image, args=(image_path,)).start()
    
def process_image(image_path):
    image_selected = Image.open(image_path)
    process_frame(image_selected, "image")
    
    # Once processing is complete, inform the user and close the dialog
    close_processing_dialog()
    messagebox.showinfo("Processing complete", "The selected action has been completed!")

def choose_video():
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mkv *.mp4 *.avi *.mov *.wmv")])
    if video_path == "":
        messagebox.showinfo("User information", "No file has been selected, the operation has been cancelled!")
        pass
    else:
        show_processing_dialog()
    
        # Start the processing task in a separate thread, passing the necessary information as arguments
        threading.Thread(target=process_video, args=(video_path, )).start()


def process_video(video_path):
    global capturing
    capturing = True  # Set capturing to True when starting the capture

    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if capturing is False:
            break

        # Convert the BGR frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convert the OpenCV image to a PIL Image
        pil_frame = Image.fromarray(frame_rgb)
        resized_frame = resize_frame(pil_frame)
        process_frame(resized_frame, "video")

    # Once processing is complete, inform the user and close the dialog
    close_processing_dialog()
    messagebox.showinfo("Processing complete", "The selected action has been completed or manually stopped!")

    
def back_to_main():
    global capturing
    capturing = False
    
    options_page.withdraw()
    root.deiconify()
    
def on_sys_btn_click(choice):
    global user_choice
    user_choice = choice
    open_options()

def blend_image_with_color(image_path, color, size=(193, 54)):
    image = Image.open(image_path).convert("RGBA")
    image = image.resize(size, Image.LANCZOS)  # Anti-aliasing applied here
    blended_image = Image.new("RGBA", size, color)
    blended_image.paste(image, (0, 0), image)
    return ImageTk.PhotoImage(blended_image)

def animate():
    global gif_index, animation_id
    try:
        if dialog.winfo_exists():                                         # Check if the dialog still exists
            gif_index = (gif_index + 1) % len(frames)
            label_image.config(image=frames[gif_index])
            animation_id = dialog.after(15, animate)
        else:
            dialog.after_cancel(animation_id)
            animation_id = None
    except _tkinter.TclError:
        pass

def show_processing_dialog():
    global dialog, label_image, frames, gif_index, animation_id
    animation_id = None
    gif_index = 0
    dialog = tk.Toplevel(root)
    dialog.geometry("500x500")
    dialog.title("Processing... Please wait!")
    dialog.iconbitmap(icon_path)          # Set the icon for the main page
    
    # Open the GIF and extract the frames
    image = Image.open('ui/processing/original.gif') # Correct the path if necessary
    frames = [ImageTk.PhotoImage(frame.copy().convert('RGBA')) for frame in ImageSequence.Iterator(image)]
    image.close()

    label_image = tk.Label(dialog, image=frames[0])
    label_image.pack(expand=True)

    # Start the animation
    animate()

def close_processing_dialog():
    try: 
        dialog.destroy()
    except _tkinter.TclError:
        pass
    
    clean_memory()
    
def speak_text(text):
    engine = pyttsx3.init()
    
    
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate + 10)        # increase the speech rate (words per minute)
    
    engine.say(text)
    engine.runAndWait()                          # blocks while processing all the currently queued commands

    #time.sleep(1)                               # ensure there's a pause between each piece of text

In [ ]:
###############################################################################################################################################
root = tk.Tk()
root.title('Plot Data Extraction via Computer Vision')
root.geometry("970x450")  # Set default size to 970x450
root.resizable(False, False) # Make the window non-resizable

original_img = Image.open("ui/backgrounds/page_1_background.png")
resized_img = original_img.resize((970, 450), Image.LANCZOS) # Resize the image
img = ImageTk.PhotoImage(resized_img)

background_label = tk.Label(root, image=img)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a frame to hold the buttons
background_color = (0,9,64,255)

button_frame = tk.Frame(root, bg="#000940")
button_frame.place(relx=0.5, rely=0.555, anchor=tk.CENTER)

main_btn1_img = blend_image_with_color("ui/buttons/system_options.png", background_color)
main_btn2_img = blend_image_with_color("ui/buttons/system_details.png", background_color)
main_btn3_img = blend_image_with_color("ui/buttons/close_system.png", background_color)

# Create buttons within the frame
main_btn1 = tk.Button(button_frame, image=main_btn1_img, command=open_system, borderwidth=0, relief='flat', highlightthickness=0)
main_btn1.grid(row=0, column=0, pady=10)

main_btn2 = tk.Button(button_frame, image=main_btn2_img, command=show_description, borderwidth=0, relief='flat', highlightthickness=0)
main_btn2.grid(row=1, column=0, pady=10)

main_btn3 = tk.Button(button_frame, image=main_btn3_img, command=quit_program, borderwidth=0, relief='flat', highlightthickness=0)
main_btn3.grid(row=2, column=0, pady=10)

###############################################################################################################################################
system_page = tk.Toplevel(root)
system_page.title('System Page')
system_page.geometry("970x450")  # Set default size to 970x450
system_page.resizable(False, False) # Make the window non-resizable
system_page.withdraw()

system_img = img

system_background_label = tk.Label(system_page, image=system_img)
system_background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a frame to hold the system buttons
system_button_frame = tk.Frame(system_page, bg="#000940")
system_button_frame.place(relx=0.5, rely=0.555, anchor=tk.CENTER)

img_sys_btn1 = blend_image_with_color("ui/buttons/plot_to_speech.png", background_color)
img_sys_btn2 = blend_image_with_color("ui/buttons/dataset_creation.png", background_color)
img_sys_btn3 = blend_image_with_color("ui/buttons/report_creation.png", background_color)

# Create buttons within the system frame
sys_btn1 = tk.Button(system_button_frame, image=img_sys_btn1, command=lambda: on_sys_btn_click("plot_to_speech"), borderwidth=0, relief='flat', highlightthickness=0)
sys_btn1.grid(row=0, column=0, pady=10)

sys_btn2 = tk.Button(system_button_frame, image=img_sys_btn2, command=lambda: on_sys_btn_click("dataset_cration"), borderwidth=0, relief='flat', highlightthickness=0)
sys_btn2.grid(row=1, column=0, pady=10)

sys_btn3 = tk.Button(system_button_frame, image=img_sys_btn3, command=lambda: on_sys_btn_click("report_creation"), borderwidth=0, relief='flat', highlightthickness=0)
sys_btn3.grid(row=2, column=0, pady=10)

###############################################################################################################################################
options_page = tk.Toplevel(root)
options_page.title('Options Page')
options_page.geometry("970x450")  # Set default size to 970x450
options_page.resizable(False, False) # Make the window non-resizable
options_page.withdraw()

options_img = img

background_label2 = tk.Label(options_page, image=options_img)
background_label2.place(x=0, y=0, relwidth=1, relheight=1)

# Create a frame to hold the options buttons on the options page
options_button_frame = tk.Frame(options_page, bg="#000940")
options_button_frame.place(relx=0.5, rely=0.555, anchor=tk.CENTER)

img_opt_btn1 = blend_image_with_color("ui/buttons/choose_picture.png", background_color)
img_opt_btn2 = blend_image_with_color("ui/buttons/choose_video.png", background_color)
img_opt_btn3 = blend_image_with_color("ui/buttons/desktop_capture.png", background_color)
img_opt_btn4 = blend_image_with_color("ui/buttons/main_window.png", background_color)

# Create buttons within the options frame
opt_btn1 = tk.Button(options_button_frame, image=img_opt_btn1, command=choose_image, borderwidth=0, relief='flat', highlightthickness=0)
opt_btn1.grid(row=0, column=0, pady=10)

opt_btn2 = tk.Button(options_button_frame, image=img_opt_btn2, command=choose_video, borderwidth=0, relief='flat', highlightthickness=0)
opt_btn2.grid(row=0, column=1, pady=10)

opt_btn3 = tk.Button(options_button_frame, image=img_opt_btn3, command=desktop_capture, borderwidth=0, relief='flat', highlightthickness=0)
opt_btn3.grid(row=1, column=0, pady=10)

opt_btn4 = tk.Button(options_button_frame, image=img_opt_btn4, command=back_to_main, borderwidth=0, relief='flat', highlightthickness=0)
opt_btn4.grid(row=1, column=1, pady=10)

###############################################################################################################################################
icon_path = "ui/window_icons/bar_icon_inverted.ico"

root.iconbitmap(icon_path)          # Set the icon for the main page
system_page.iconbitmap(icon_path)   # Set the icon for the system page
options_page.iconbitmap(icon_path)  # Set the icon for the options page

###############################################################################################################################################
root.mainloop()
